<a href="https://colab.research.google.com/github/HYUNMIN-HWANG/Colab/blob/main/DACON_vision2/DACON_vision2_0213_1_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 실행시 등장하는 URL을 클릭하여 허용해주면 인증KEY가 나타난다. 복사하여 URL아래 빈칸에 붙여넣으면 마운트에 성공하게된다.
from google.colab import drive
drive.mount('./MyDrive')

Mounted at ./MyDrive


In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


######################################################
# File Load
dataset = pd.read_csv('./MyDrive/MyDrive/Colab/DACON_vision2/dirty_mnist_2nd_answer.csv')
y_df =  dataset.iloc[:,:]
print(y_df.shape)  # (50000, 27)

sub = pd.read_csv('./MyDrive/MyDrive/Colab/DACON_vision2/sample_submission.csv')
print(sub.shape)    # (5000, 27)

######################################################

train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=5,
    fill_mode='nearest'
)
etc_datagen = ImageDataGenerator(rescale=1./255)

def mymodel () :
    model = Sequential()
    model.add(Conv2D(32, (2,2), padding='same', input_shape=(80, 80, 1), activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (2,2), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(64, (2,2), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (2,2), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dense(1, activation='sigmoid'))

    return model

#1. DATA
x = np.load('./MyDrive/MyDrive/Colab/DACON_vision2/npy/vision_x4.npy')
x_pred = np.load('./MyDrive/MyDrive/Colab/DACON_vision2/npy/vision_x_pred4.npy')
print("<==complete load==>")

print(x.shape, x_pred.shape) # (50000, 80, 80, 1) (5000, 80, 80, 1)
x[x < 253] = 0

alphabet = ['a']

val_loss_list = []
val_acc_list = []

for alph in alphabet :  # 한 알파벳씩 검증
    print('<<<<<<<<<<<< ', alph, ' predict start >>>>>>>>>>>>')
    y = y_df.loc[:,alph]
    y_pred_list = []
    y_result = 0

    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, shuffle=True, random_state=42)
    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, train_size=0.8, shuffle=True, random_state=42)
    print(x_train.shape, x_test.shape, x_valid.shape)  # (32000, 80, 80, 1) (10000, 80, 80, 1) (8000, 80, 80, 1)
    print(y_train.shape, y_test.shape, y_valid.shape)  # (32000,) (10000,) (8000,)

    batch = 20
    train_generator = train_datagen.flow(x_train, y_train, batch_size=batch, seed=2021)
    test_generator = etc_datagen.flow(x_test, y_test, batch_size=batch, seed=2021)
    valid_generator = etc_datagen.flow(x_valid, y_valid)
    pred_generator = etc_datagen.flow(x_pred)

    #2. Modeling
    model = mymodel()

    #3. Compile, Train

    path = './MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0213_1_a.hdf5'
    cp = ModelCheckpoint(filepath=path, monitor='val_loss', mode='min', save_best_only=True, verbose=1)
    es = EarlyStopping(monitor='val_loss', patience=30, mode='min')
    lr = ReduceLROnPlateau(monitor='val_loss', patience=15, factor=0.4, mode='min')

    model.compile(
        loss='binary_crossentropy', 
        optimizer=Adam(lr=0.05, epsilon=None), 
        metrics=['acc']
    )

    hist = model.fit_generator(
        train_generator, epochs=50,
        steps_per_epoch = len(x_train) // batch ,
        validation_data=valid_generator, callbacks=[es, lr, cp]
    )

    # loss가 가장 작은 것 로드
    model = load_model('./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0213_1_a.hdf5')

    #4. Evaluate, Predict
    loss, acc = model.evaluate(test_generator)
    print("loss : ", loss)
    print("acc : ", acc)  

    val_loss_list.append(loss)
    val_acc_list.append(acc)

    # loss :  
    # acc :  

    y_pred = model.predict(pred_generator)
    # print(y_pred.shape) # (5000, 1)
    y_result= np.where(y_pred < 0.5, 0, 1)
    sub.loc[:,alph] = y_result  # 예측값을 sub 파일에 저장
    print(sub.head())

    print("- ", alph, " predict end -\n")

print("mean loss : ", sum(val_loss_list)/len(alphabet))
print("mean acc : ", sum(val_acc_list)/len(alphabet))

sub.to_csv('./MyDrive/MyDrive/Colab/DACON_vision2/sub_0213_1_a.csv', index=False)
print(sub.head())

# sub
# score 

(50000, 27)
(5000, 27)
<==complete load==>
(50000, 80, 80, 1) (5000, 80, 80, 1)
<<<<<<<<<<<<  a  predict start >>>>>>>>>>>>
(32000, 80, 80, 1) (10000, 80, 80, 1) (8000, 80, 80, 1)
(32000,) (10000,) (8000,)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
1600/1600 [==============================] - 33s 17ms/step - loss: 0.7047 - acc: 0.5235 - val_loss: 0.6959 - val_acc: 0.4845

Epoch 00001: val_loss improved from inf to 0.69592, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0213_1_a.hdf5
Epoch 2/50
1600/1600 [==============================] - 27s 17ms/step - loss: 0.6925 - acc: 0.5341 - val_loss: 163.3115 - val_acc: 0.4609

Epoch 00002: val_loss did not improve from 0.69592
Epoch 3/50
1600/1600 [==============================] - 26s 16ms/step - loss: 0.6943 - acc: 0.5344 - val_loss: 0.6906 - val_acc: 0.5359

Epoch 00003: val_loss improved from 0.69592 to 0.69057, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0213_1_a.hdf5
Epoch 4/50
1600/1600 [==============================] - 27s 17ms/step - loss: 0.6934 - acc: 0.5327 - val_loss: 0.6956 - val_acc: 0.4750

Epoch 00004: val_loss did not improve from 0.69057
Epoch 5/50
1600/1600 [==============================] - 26s 16ms/step - loss: 0.6951

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


######################################################
# File Load
dataset = pd.read_csv('./MyDrive/MyDrive/Colab/DACON_vision2/dirty_mnist_2nd_answer.csv')
y_df =  dataset.iloc[:,:]
print(y_df.shape)  # (50000, 27)

sub = pd.read_csv('./MyDrive/MyDrive/Colab/DACON_vision2/sample_submission.csv')
print(sub.shape)    # (5000, 27)

######################################################

train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=5,
    fill_mode='nearest'
)
etc_datagen = ImageDataGenerator(rescale=1./255)

def mymodel () :
    model = Sequential()
    model.add(Conv2D(32, (2,2), padding='same', input_shape=(80, 80, 1), activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(AveragePooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (2,2), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(AveragePooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (2,2), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(AveragePooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dense(1, activation='sigmoid'))

    return model

#1. DATA
x = np.load('./MyDrive/MyDrive/Colab/DACON_vision2/npy/vision_x4.npy')
x_pred = np.load('./MyDrive/MyDrive/Colab/DACON_vision2/npy/vision_x_pred4.npy')
print("<==complete load==>")

print(x.shape, x_pred.shape) # (50000, 80, 80, 1) (5000, 80, 80, 1)
x[x < 253] = 0

# alphabet = ['a','b','c','d','e','f','g','h','i','j','k']
# alphabet = ['l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

val_loss_list = []
val_acc_list = []

for alph in alphabet :  # 한 알파벳씩 검증
    print('<<<<<<<<<<<< ', alph, ' predict start >>>>>>>>>>>>')
    y = y_df.loc[:,alph]
    y_pred_list = []
    y_result = 0

    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, shuffle=True, random_state=42)
    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, train_size=0.8, shuffle=True, random_state=42)
    print(x_train.shape, x_test.shape, x_valid.shape)  # (32000, 80, 80, 1) (10000, 80, 80, 1) (8000, 80, 80, 1)
    print(y_train.shape, y_test.shape, y_valid.shape)  # (32000,) (10000,) (8000,)

    batch = 50
    train_generator = train_datagen.flow(x_train, y_train, batch_size=batch, seed=2021)
    test_generator = etc_datagen.flow(x_test, y_test, batch_size=batch, seed=2021)
    valid_generator = etc_datagen.flow(x_valid, y_valid)
    pred_generator = etc_datagen.flow(x_pred)

    #2. Modeling
    model = mymodel()

    #3. Compile, Train

    path = './MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_{val_loss:.4f}.hdf5'
    cp = ModelCheckpoint(filepath=path, monitor='val_loss', mode='min', save_best_only=True, verbose=1)
    es = EarlyStopping(monitor='val_loss', patience=30, mode='min')
    lr = ReduceLROnPlateau(monitor='val_loss', patience=15, factor=0.4, mode='min')

    model.compile(
        loss='binary_crossentropy', 
        optimizer=Adam(lr=0.05, epsilon=None), 
        metrics=['acc']
    )

    hist = model.fit_generator(
        train_generator, epochs=50,
        steps_per_epoch = len(x_train) // batch ,
        validation_data=valid_generator, callbacks=[es, lr, cp]
    )

    #4. Evaluate, Predict
    loss, acc = model.evaluate(test_generator)
    print("loss : ", loss)
    print("acc : ", acc)  

    val_loss_list.append(loss)
    val_acc_list.append(acc)

    # mean loss :  3280.4362607677776
    # mean acc :  0.5196333249409993

    y_pred = model.predict(pred_generator)
    # print(y_pred.shape) # (5000, 1)
    y_result= np.where(y_pred < 0.5, 0, 1)
    sub.loc[:,alph] = y_result  # 예측값을 sub 파일에 저장
    print(sub.head())

    print("- ", alph, " predict end -\n")

print("mean loss : ", sum(val_loss_list)/len(alphabet))
print("mean acc : ", sum(val_acc_list)/len(alphabet))

sub.to_csv('./MyDrive/MyDrive/Colab/DACON_vision2/sub_0212_1_2.csv', index=False)
print(sub.head())

# sub
# score 

(50000, 27)
(5000, 27)
<==complete load==>
(50000, 80, 80, 1) (5000, 80, 80, 1)
<<<<<<<<<<<<  l  predict start >>>>>>>>>>>>
(32000, 80, 80, 1) (10000, 80, 80, 1) (8000, 80, 80, 1)
(32000,) (10000,) (8000,)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
640/640 [==============================] - 25s 32ms/step - loss: 0.7144 - acc: 0.5303 - val_loss: 0.7149 - val_acc: 0.5361

Epoch 00001: val_loss improved from inf to 0.71488, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.7149.hdf5
Epoch 2/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6920 - acc: 0.5296 - val_loss: 40.7898 - val_acc: 0.5077

Epoch 00002: val_loss did not improve from 0.71488
Epoch 3/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6943 - acc: 0.5249 - val_loss: 0.7130 - val_acc: 0.4608

Epoch 00003: val_loss improved from 0.71488 to 0.71303, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.7130.hdf5
Epoch 4/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6938 - acc: 0.5224 - val_loss: 0.6935 - val_acc: 0.5135

Epoch 00004: val_loss improved from 0.71303 to 0.69347, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.6935.

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
640/640 [==============================] - 21s 31ms/step - loss: 0.7108 - acc: 0.5280 - val_loss: 0.7029 - val_acc: 0.4666

Epoch 00001: val_loss improved from inf to 0.70293, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.7029.hdf5
Epoch 2/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6930 - acc: 0.5304 - val_loss: 0.6938 - val_acc: 0.5328

Epoch 00002: val_loss improved from 0.70293 to 0.69376, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.6938.hdf5
Epoch 3/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6928 - acc: 0.5359 - val_loss: 0.6911 - val_acc: 0.5378

Epoch 00003: val_loss improved from 0.69376 to 0.69113, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.6911.hdf5
Epoch 4/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6930 - acc: 0.5299 - val_loss: 0.7137 - val_acc: 0.5206

Epoch 00004: val_loss did not improve from 0.69

...

Epoch 1/50
640/640 [==============================] - 22s 32ms/step - loss: 0.7080 - acc: 0.5272 - val_loss: 0.6901 - val_acc: 0.5468

Epoch 00001: val_loss improved from inf to 0.69011, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.6901.hdf5
Epoch 2/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6940 - acc: 0.5310 - val_loss: 0.8930 - val_acc: 0.5468

Epoch 00002: val_loss did not improve from 0.69011
Epoch 3/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6931 - acc: 0.5297 - val_loss: 0.6892 - val_acc: 0.5469

Epoch 00003: val_loss improved from 0.69011 to 0.68922, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.6892.hdf5
Epoch 4/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6922 - acc: 0.5343 - val_loss: 4.9489 - val_acc: 0.5468

Epoch 00004: val_loss did not improve from 0.68922
Epoch 5/50
640/640 [==============================] - 20s 32ms/step - loss: 0.6934 -

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
640/640 [==============================] - 22s 32ms/step - loss: 0.7090 - acc: 0.5238 - val_loss: 0.6947 - val_acc: 0.4694

Epoch 00001: val_loss improved from inf to 0.69465, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.6947.hdf5
Epoch 2/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6936 - acc: 0.5272 - val_loss: 0.6911 - val_acc: 0.5331

Epoch 00002: val_loss improved from 0.69465 to 0.69107, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.6911.hdf5
Epoch 3/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6922 - acc: 0.5379 - val_loss: 0.6922 - val_acc: 0.5310

Epoch 00003: val_loss did not improve from 0.69107
Epoch 4/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6912 - acc: 0.5371 - val_loss: 0.6914 - val_acc: 0.5331

Epoch 00004: val_loss did not improve from 0.69107
Epoch 5/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6942 -

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
640/640 [==============================] - 22s 32ms/step - loss: 0.7112 - acc: 0.5320 - val_loss: 1.7586 - val_acc: 0.5307

Epoch 00001: val_loss improved from inf to 1.75856, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_1.7586.hdf5
Epoch 2/50
640/640 [==============================] - 20s 32ms/step - loss: 0.6931 - acc: 0.5312 - val_loss: 0.7638 - val_acc: 0.5344

Epoch 00002: val_loss improved from 1.75856 to 0.76378, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.7638.hdf5
Epoch 3/50
640/640 [==============================] - 20s 31ms/step - loss: 0.6931 - acc: 0.5308 - val_loss: 0.6942 - val_acc: 0.4818

Epoch 00003: val_loss improved from 0.76378 to 0.69421, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.6942.hdf5
Epoch 4/50
640/640 [==============================] - 20s 32ms/step - loss: 0.6944 - acc: 0.5281 - val_loss: 0.6949 - val_acc: 0.5344

Epoch 00004: val_loss did not improve from 0.69

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


######################################################
# File Load
dataset = pd.read_csv('./MyDrive/MyDrive/Colab/DACON_vision2/dirty_mnist_2nd_answer.csv')
y_df =  dataset.iloc[:,:]
print(y_df.shape)  # (50000, 27)

sub = pd.read_csv('./MyDrive/MyDrive/Colab/DACON_vision2/sample_submission.csv')
print(sub.shape)    # (5000, 27)

######################################################

train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=5,
    fill_mode='nearest'
)
etc_datagen = ImageDataGenerator(rescale=1./255)

def mymodel () :
    model = Sequential()
    model.add(Conv2D(32, (2,2), padding='same', input_shape=(80, 80, 1), activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(AveragePooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (2,2), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(AveragePooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (2,2), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(AveragePooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dense(1, activation='sigmoid'))

    return model

#1. DATA
x = np.load('./MyDrive/MyDrive/Colab/DACON_vision2/npy/vision_x4.npy')
x_pred = np.load('./MyDrive/MyDrive/Colab/DACON_vision2/npy/vision_x_pred4.npy')
print("<==complete load==>")

print(x.shape, x_pred.shape) # (50000, 80, 80, 1) (5000, 80, 80, 1)
x[x < 253] = 0

# alphabet = ['a','b','c','d','e','f','g','h','i','j','k']
# alphabet = ['l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
alphabet = ['b','c','f','g','h','i','j','k','l','n','p','q','r','u','v','w']

val_loss_list = []
val_acc_list = []

for alph in alphabet :  # 한 알파벳씩 검증
    print('<<<<<<<<<<<< ', alph, ' predict start >>>>>>>>>>>>')
    y = y_df.loc[:,alph]
    y_pred_list = []
    y_result = 0

    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, shuffle=True, random_state=42)
    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, train_size=0.8, shuffle=True, random_state=42)
    print(x_train.shape, x_test.shape, x_valid.shape)  # (32000, 80, 80, 1) (10000, 80, 80, 1) (8000, 80, 80, 1)
    print(y_train.shape, y_test.shape, y_valid.shape)  # (32000,) (10000,) (8000,)

    batch = 50
    train_generator = train_datagen.flow(x_train, y_train, batch_size=batch, seed=2021)
    test_generator = etc_datagen.flow(x_test, y_test, batch_size=batch, seed=2021)
    valid_generator = etc_datagen.flow(x_valid, y_valid)
    pred_generator = etc_datagen.flow(x_pred)

    #2. Modeling
    # model = mymodel()
    model = load_model('./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_0.6878.hdf5')

    #3. Compile, Train

    # path = './MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0212_1_{val_loss:.4f}.hdf5'
    # cp = ModelCheckpoint(filepath=path, monitor='val_loss', mode='min', save_best_only=True, verbose=1)
    # es = EarlyStopping(monitor='val_loss', patience=30, mode='min')
    # lr = ReduceLROnPlateau(monitor='val_loss', patience=15, factor=0.4, mode='min')

    # model.compile(
    #     loss='binary_crossentropy', 
    #     optimizer=Adam(lr=0.05, epsilon=None), 
    #     metrics=['acc']
    # )

    # hist = model.fit_generator(
    #     train_generator, epochs=50,
    #     steps_per_epoch = len(x_train) // batch ,
    #     validation_data=valid_generator, callbacks=[es, lr, cp]
    # )

    #4. Evaluate, Predict
    loss, acc = model.evaluate(test_generator)
    print("loss : ", loss)
    print("acc : ", acc)  

    val_loss_list.append(loss)
    val_acc_list.append(acc)

    # mean loss :  3280.4362607677776
    # mean acc :  0.5196333249409993

    y_pred = model.predict(pred_generator)
    # print(y_pred.shape) # (5000, 1)
    y_result= np.where(y_pred < 0.5, 0, 1)
    sub.loc[:,alph] = y_result  # 예측값을 sub 파일에 저장
    print(sub.head())

    print("- ", alph, " predict end -\n")

print("mean loss : ", sum(val_loss_list)/len(alphabet))
print("mean acc : ", sum(val_acc_list)/len(alphabet))

sub.to_csv('./MyDrive/MyDrive/Colab/DACON_vision2/sub_0212_1_3.csv', index=False)
print(sub.head())

# sub
# score 

(50000, 27)
(5000, 27)
<==complete load==>
(50000, 80, 80, 1) (5000, 80, 80, 1)
<<<<<<<<<<<<  b  predict start >>>>>>>>>>>>
(32000, 80, 80, 1) (10000, 80, 80, 1) (8000, 80, 80, 1)
(32000,) (10000,) (8000,)
200/200 [==============================] - 1s 6ms/step - loss: 0.6902 - acc: 0.5342
loss :  0.6901840567588806
acc :  0.5342000126838684
   index  a  b  c  d  e  f  g  h  i  j  k  ...  o  p  q  r  s  t  u  v  w  x  y  z
0  50000  0  0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0  0  0
1  50001  0  1  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0  0  0
2  50002  0  1  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0  0  0
3  50003  0  0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0  0  0
4  50004  0  0  0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  0  0  0  0

[5 rows x 27 columns]
-  b  predict end -

<<<<<<<<<<<<  c  predict start >>>>>>>>>>>>
(32000, 80, 80, 1) (10000, 80, 80, 1) (8000, 80, 80, 1)
(32000,) (10000

In [ ]:
# Modelcheckpoint에서 loss 가장 작은 거 넣었더니 모든 알파벳에서 같은 예측결과가 나온다.
# 알파벳 마다 다 다르게 모델을 해서 저장을 해야 하는 건가.... (어느 세월에 ?? )
# sub_0212_1.csv
# score 0.5084307692